<a href="https://colab.research.google.com/github/Hemul997/Hyperparams-Optimisation/blob/master/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 1.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=e60d67727ce090e70b23f155d815a31a57e0707b1a934edef9dc17e316bd7324
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=7d594b882fed3c0da07f6ca5e82de46927780b557380e920472e7c40211f92a1
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow import keras

from keras.datasets import mnist

import kerastuner as kt

from kerastuner.engine.hyperparameters import HyperParameters


num_classes = 10


(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test  = x_test.reshape(10000, 784)


x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
    
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)    

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
"""
tuner = kt.Hyperband(create_demo_model,
                     objective = 'val_accuracy', 
                     max_epochs = 20,
                     hyperband_iterations = 5,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt') 

tuner.search(x_train, y_train, epochs = 20, validation_data = (x_test, y_test))
"""                

In [ ]:
num_classes = 10

global_params = {
    'layers'      : tuple(range(2, 4)), #Global layers
    'batch_size'  : tuple(range(8, 32, 8)),
    'epochs'      : tuple(range(1, 5)),
    'optimizers'  : ('SGD', 'Adam', 'Adagrad', 'RMSprop', 'Adadelta', 'Adamax', 'Nadam'),
    'learning_rate'  : tuple(np.logspace(np.log10(0.005), np.log10(0.5), base = 10, num = 100))
}


local_params = {

    'neurals'     : tuple(range(128, 512, 128)), #Neurals on each layer
    'func_act'    : ( 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'softsign', 'hard_sigmoid', 'linear', 'exponential' ),
    'func_reg'    : ( 'None', 'l1', 'l2', 'l1_l2' ),
    'reg_coef'    :  tuple(np.logspace(np.log10(1e-7), np.log10(1e-4), base = 10, num = 100)),
    'weight_init' : ( 'random_normal', 'random_uniform', 'truncated_normal', 'glorot_normal', 'glorot_uniform'
                     , 'identity', 'orthogonal' )
}

def init_regularizer(func_reg, coef_reg):
    if (func_reg == 'None'):
        return func_reg
    
    regularizer = 'tf.keras.regularizers.' + func_reg + '(' + str(coef_reg)
    
    if (func_reg == 'l1_l2') :
        regularizer += ', ' + str(coef_reg)
        
    regularizer += ')'
    
    return regularizer


def create_demo_model(hp):
    network = keras.Sequential()      
    
    for i in range(hp.Choice('num_layers', global_params['layers'])):
        
        func_reg = hp.Choice('func_reg_' + str(i), local_params['func_reg'])
        coef_reg = hp.Choice('reg_coef_' + str(i),  local_params['reg_coef'])
        regularizer = init_regularizer(func_reg, coef_reg)
        network.add(tf.keras.layers.Dense(units=hp.Choice('units_' + str(i), local_params['neurals']), 
                                          activation=hp.Choice('func_act_' + str(i), local_params['func_act']),
                                          kernel_regularizer=eval(regularizer),
                                          kernel_initializer = hp.Choice('weight_init', local_params['weight_init'])))
        
    network.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    
    hp_learning_rate = hp.Choice('learning_rate', values = global_params['learning_rate'])
    
    hp_optimizer = 'tf.keras.optimizers.' + hp.Choice('func_opt', global_params['optimizers']) + '(' + str(hp_learning_rate) + ')';
    
    network.compile(optimizer = eval(hp_optimizer), loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    return network


class CustomHyperband(kt.tuners.Hyperband):
    def run_trial(self, trial, *args, **kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops
        # via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 32, 256, step=32)
        kwargs['epochs'] = trial.hyperparameters.Int('epochs', 10, 30)
        super(CustomHyperband, self).run_trial(trial, *args, **kwargs)
    
class CustomBayesianOptimisation(kt.tuners.BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops
        # via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 32, 256, step=32)
        kwargs['epochs'] = trial.hyperparameters.Int('epochs', 10, 30)
        super(CustomBayesianOptimisation, self).run_trial(trial, *args, **kwargs)

In [ ]:
hyperband_tuner = CustomHyperband(create_demo_model,
                              objective = 'val_accuracy', 
                              max_epochs = 20,
                              hyperband_iterations = 5,
                              factor = 3,
                              directory = 'my_dir',
                              project_name = 'intro_to_kt')


#Max_trials - количество экспериментов
bayesian_tuner = CustomBayesianOptimisation(create_demo_model,
                                           objective = 'val_accuracy',
                                           max_trials = 10,
                                           directory = 'my_dir',
                                           project_name = 'intro_to_bayesian')


#hyperband_tuner.search(x_train, y_train, validation_data = (x_test, y_test)
#            , callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy', patience=3)])

bayesian_tuner.search(x_train, y_train, validation_data = (x_test, y_test))

In [ ]:
hyperband_tuner = CustomHyperband(create_demo_model,
                              objective = 'val_accuracy', 
                              max_epochs = 20,
                              hyperband_iterations = 5,
                              factor = 3)


